In [1]:
import time
import duckdb

import load_to_db
import embeddings
import db_queries
import main




# this is the file to which the sql database will be saved
db_file_c = duckdb.connect("/cluster/work/coss/anmusso/victoria/loaded_data/loaded_comments.db")
db_file_s = duckdb.connect("/cluster/work/coss/anmusso/victoria/loaded_data/loaded_submissions.db")



(We could consider changing extract_comments to also add to existing tables)

Load comments into duckdb and get embeddings 

In [ ]:


main.load('S', '05', '2020')
#main.load('C', '05', '2020')
#main.load('S', '06', '2020')
#main.load('C', '06', '2020')

In [2]:
#run to check number of submissions/comments loaded
start = time.time()
print("C5: \n", db_file_c.sql("SELECT Count(*) FROM C_05_2020;"))
print("S5: \n", db_file_s.sql('SELECT Count(*) FROM S_05_2020;'))
print("C6: \n", db_file_c.sql('SELECT Count(*) FROM C_06_2020;'))
print("S6: \n", db_file_s.sql('SELECT Count(*) FROM S_06_2020;'))
print("time to compute: ", time.time()-start)

C5: 
 ┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     25682568 │
└──────────────┘

S5: 
 ┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       710071 │
└──────────────┘

C6: 
 ┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     37746897 │
└──────────────┘

S6: 
 ┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1017459 │
└──────────────┘

time to compute:  0.18347382545471191


Calculate embeddings: (uncomment line)
available models: 'all-mpnet-base-v2' , 'all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1'
will automatically use gpu if available
use main.py with batch system to choose GPU

In [4]:
db_file_s.sql(f"SELECT * FROM {main.table_id('S','05','2020')} LIMIT 10").fetchdf()

,NUM,subreddit,score,author,created_utc,title,id,num_comments,upvote_ratio
0,0,aaaaaaacccccccce,68,[deleted],1588291203,LMAO the ace post and the one at the bottom be...,gb7h31,16,0.97
1,1,SuicideWatch,78,awayitsthrownnn,1588291204,Getting downvoted for posting my feelings on t...,gb7h3f,17,0.87
2,2,animalid,62,Feanux,1588291205,White Muscovy duck or another kind?,gb7h3w,12,0.98
3,3,anime,78,AnimeMod,1588291205,"Casual Discussion Fridays - Week of May 01, 2020",gb7h42,11002,0.95
4,4,SkyGame,297,aiden_saxon,1588291207,Made some friends gathering krabs,gb7h50,12,1.00
5,5,realasians,931,littlesubgirl_,1588291209,"Good morning, here is a nude ;) [OC]",gb7h5t,11,0.97
6,6,KimetsuNoYaiba,290,CheetahSperm18,1588291209,Tanjirou &amp; the gang trying their luck with...,gb7h5w,11,0.99
7,7,regina,96,smell-the-glove,1588291210,Starting tomorrow Caraway Grill is open again ...,gb7h6i,26,0.94
8,8,BeardedDragons,31,[deleted],1588291211,My dude enjoying his worm buffet.,gb7h6t,16,0.93
9,9,airsoft,27,MrItalianGamer,1588291214,I recently saw people asking about an official...,gb7h84,22,0.95


In [ ]:
t,m,y = 'S','06','2020' #type month and year
model_name = 'all-MiniLM-L6-v2'


start = time.time()
#num = main.embedd(m, y, t, model_name=model_name) #returns number of embeddings calculated

with open('time_tracking_2.txt', 'a') as f: #loggs time into file for comparison
    f.write(f"set: {m+t+y}, size:{num} model:{model_name}, gpu:{torch.cuda.get_device_name(0)} , time:{time.time()-start} \n")


Retrieve all entry numbers for certain subreddits within given table

In [5]:
t,m,y = 'S','06','2020' #type month and year
subr = ['Republican', 'democrats','healthcare', 'Feminism', 'nra', 'education', 'climatechange',
            'politics','progressive', 'The_Donald','TrueChristian','Trucks','teenagers','AskMenOver30',
            'backpacking','news','BlackLivesMatter','racism','news','usa','DefundPoliceNYC']

subr_nums = db_queries.get_subreddit_numbers(table_name = main.table_id(t,m,y), sql_db = db_file_s, subreddits = subr)
print(len(subr_nums))

20758


load embeddings of retrieved numbers

In [6]:
M_embedd = main.fetch_embeddings(num_list = subr_nums, table_name=main.table_id(t,m,y), model_name= 'all-MiniLM-L6-v2')
print(M_embedd.shape)
#print(M_embedd[0])

(20758, 384)


now we can umap the embeddings

In [7]:
import embedding_graphs.UMAP_embeddings as umap

atr = ['title', 'subreddit']
atr_df = db_queries.get_attributes(attributes=atr, numbers=subr_nums, table_name=main.table_id(t,m,y), sql_db=db_file_s)
print(atr_df)
umap.show_umap(M_embedd,atr_df,'umap try 1')

/cluster/apps/nss/gcc-8.2.0/python/3.11.2/x86_64/lib64/python3.11/site-packages/umap_learn-0.5.3-py3.11.egg/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/cluster/apps/nss/gcc-8.2.0/python/3.11.2/x86_64/lib64/python3.11/site-packages/umap_learn-0.5.3-py3.11.egg/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for detail

                                                   title  subreddit
0      Trump's crazy designation of Antifa as terrori...   politics
1      Will there be a second stimulus check for Amer...   politics
2      Thousands march in Britain in solidarity with ...       news
3             Rumsfeld: Looting is transition to freedom   politics
4      Former S.C. Secessionist Party founder apologi...       news
...                                                  ...        ...
20753  Target employee says N.Y. customer demanded sh...       news
20754  Democrats say they have a bold climate plan – ...   politics
20755  Obama reportedly said Trump’s use of ‘kung flu...   politics
20756  Just a small little (better looking) recreatio...  teenagers
20757  As Coronavirus Pandemic Persists, Mitch McConn...   politics

[20758 rows x 2 columns]
transformed and fit
